In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import requests

In [2]:
tg_api_token = '5102696637:AAHcm9cOmIF7-GyG7nHHJYA_yZCGSWSTu60'
tg_chat_id = '1126129331'

def send_tg_message(text='Cell execution completed.'):
    requests.post(
        'https://api.telegram.org/' +
        'bot{}/sendMessage'.format(tg_api_token), 
        params=dict(chat_id=tg_chat_id, text=text)
    )

In [3]:
def print_distributions_of_labels(labels_np):
    unique_values, counts = np.unique(labels_np, axis=0, return_counts=True)
    print(f"{unique_values} {counts}")
    print(f"{unique_values} {counts/sum(counts)}")

In [4]:
f=open("input_model_data",'rb')
data=pickle.load(f,encoding='latin1')

In [5]:
validation_features = data["validation_data_features"]
validation_targets = data["validation_data_targets"]

In [6]:
validation_features.shape, validation_targets.shape

((33066, 630, 5), (33066,))

In [7]:
print_distributions_of_labels(validation_targets)

[0 1 2 3 4] [12502  8100  4677  2479  5308]
[0 1 2 3 4] [0.3780923  0.24496462 0.14144438 0.07497127 0.16052743]


In [8]:
def remove_data(features, targets, label_id):
    indexes = np.where(targets == label_id)
    return np.delete(features, indexes, axis=0), np.delete(targets, indexes)

def update_labels(targets, old_label_id, new_label_id):
    '''baseline = 0, stress = 1, amusement = 2'''
    indexes = np.where(targets == old_label_id)
    targets[indexes] = new_label_id
    return targets

In [9]:
for label_id in [0, 4, 5]:
    validation_features, validation_targets = remove_data(validation_features, validation_targets, label_id)
print(validation_features.shape, validation_targets.shape)

for old_label_id, new_label_id in [(1, 0), (2, 1), (3, 2)]:
    validation_targets = update_labels(validation_targets, old_label_id, new_label_id)
print(validation_features.shape, validation_targets.shape)
print_distributions_of_labels(validation_targets)

(15256, 630, 5) (15256,)
(15256, 630, 5) (15256,)
[0 1 2] [8100 4677 2479]
[0 1 2] [0.53093865 0.30656791 0.16249345]


In [10]:
chest_signal_available = ['ECG', 'EMG', 'EDA', 'Temp', 'Resp']
num_features = len(chest_signal_available)

In [11]:
win_size = (90*700)//100
step_slinding_window = 100

In [12]:
current_experiment = "1,5min-1sec-60percent-100pts"

# Loading Models

In [13]:
from tensorflow.keras.models import load_model

In [14]:
models_path = os.path.join("models", current_experiment)

In [15]:
models_available = os.listdir(models_path)
models_available

['softmax_lstm_baseline_rmsprop',
 'softmax_gru_baseline_rmsprop',
 'softmax_lstm_baseline_nadam',
 'softmax_lstm_baseline_adam',
 'softmax_gru_baseline_adam',
 'softmax_gru_baseline_nadam']

In [16]:
models_to_evaluate = {}

for model_name in models_available:
    models_to_evaluate[model_name] = load_model(os.path.join(models_path, model_name))
    
models_to_evaluate

2022-01-31 11:21:51.753262: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-31 11:21:52.075235: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 561 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:41:00.0, compute capability: 7.5


{'softmax_lstm_baseline_rmsprop': <keras.engine.sequential.Sequential at 0x7f7ff4474dc0>,
 'softmax_gru_baseline_rmsprop': <keras.engine.sequential.Sequential at 0x7f7c16b29580>,
 'softmax_lstm_baseline_nadam': <keras.engine.sequential.Sequential at 0x7f7c16b29550>,
 'softmax_lstm_baseline_adam': <keras.engine.sequential.Sequential at 0x7f7c003a1250>,
 'softmax_gru_baseline_adam': <keras.engine.sequential.Sequential at 0x7f7b40472220>,
 'softmax_gru_baseline_nadam': <keras.engine.sequential.Sequential at 0x7f7b3034cfa0>}

# Evaluating Models

In [17]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef, classification_report
from tensorflow import convert_to_tensor

labels = [ "Baseline", "Stress", "Amusement" ]

pacient_state = {
    0: "Not Defined/Transient",
    1: "Baseline",
    2: "Stress",
    3: "Amusement",
    4: "Meditation",
    5: "Error/Not Labeled"
}

In [18]:
font = {'family' : 'normal',
            'weight' : 'bold',
            'size'   : 30}
plt.rc("font",**font)

y_test = validation_targets

for model_name in models_to_evaluate:
    
    model = models_to_evaluate[model_name]
    
    y_pred_prob = model.predict( x = convert_to_tensor(validation_features) )

    predictions = []
    for i in range(len(y_pred_prob)):
        predictions.append(np.argmax(y_pred_prob[i]))
    
    y_pred = np.array(predictions)

    df_y = pd.DataFrame({
        "y_test": y_test,
        "y_pred": y_pred
    })

    y_test_label = np.array([pacient_state[i] for i in y_test])
    y_pred_label = np.array([pacient_state[i] for i in y_pred])

    cm = confusion_matrix(df_y["y_test"].to_list(), df_y["y_pred"].to_list())
    cm_df = pd.DataFrame(cm, index = labels, columns = labels)
    
    print(f"\n\n############### START MODEL {model_name} ###############\n\n")

    plt.figure(figsize=(30,15))
    sns.heatmap(cm_df, annot=True, cmap="Blues", fmt="d")
    plt.title(f'{model_name} - Multi-Classification Confusion Matrix')
    plt.ylabel('Actual Values')
    plt.xlabel('Predicted Values')
    plt.show()

    print(f'Classification Report: \n\n{classification_report(df_y["y_test"], df_y["y_pred"], target_names=labels)}')
    print(f'Matthews Correlation: {matthews_corrcoef(df_y["y_test"], df_y["y_pred"])}')
    
    print(f"\n\n############### END MODEL {model_name} ###############\n\n")

2022-01-31 11:21:59.054598: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-01-31 11:21:59.126404: E tensorflow/stream_executor/dnn.cc:764] CUDNN_STATUS_EXECUTION_FAILED
in tensorflow/stream_executor/cuda/cuda_dnn.cc(2066): 'cudnnRNNForwardTraining( cudnn.handle(), rnn_desc.handle(), model_dims.max_seq_length, input_desc.handles(), input_data.opaque(), input_h_desc.handle(), input_h_data.opaque(), input_c_desc.handle(), input_c_data.opaque(), rnn_desc.params_handle(), params.opaque(), output_desc.handles(), output_data->opaque(), output_h_desc.handle(), output_h_data->opaque(), output_c_desc.handle(), output_c_data->opaque(), workspace.opaque(), workspace.size(), reserve_space.opaque(), reserve_space.size())'
2022-01-31 11:21:59.126462: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at cudnn_rnn_ops.cc:1562 : INTERNAL: Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_laye

InternalError:    Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 5, 128, 1, 630, 32, 128] 
	 [[{{node CudnnRNN}}]]
	 [[sequential_3/lstm_3/PartitionedCall]] [Op:__inference_predict_function_38898]

Function call stack:
predict_function -> predict_function -> predict_function
